In [10]:
#SCRAPING ENVOIREMENT

In [2]:
#Importing libaries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools

In [4]:
#Importing page-links
list_url = []
for page in range(2,4,1):
    url = f'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page={page}'
    list_url.append(url)

list_url





['https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=2',
 'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page=3']

In [6]:
#Retriving HTMLS
list_htmls =[]
for url in list_url:
    response = requests.get(url, headers={'name': 'Mathias Rasmussen', 'email': 'kbd655@alumni.ku.dk'})
    list_htmls.append(response.text)
    time.sleep(0.5)


    

In [8]:
# Finding links on appartments
links = []
for html in list_htmls:
    soup = BeautifulSoup(html, 'lxml')
    pattern = re.compile(r'href\":\"(/addresses/[^\"]+)')
    matches = pattern.findall(str(soup))

    apartments_list = list(set(matches))


    base_url = "https://www.boligsiden.dk"
    apartment_links = [base_url + link for link in apartments_list] #This list contains appartments
    links.append(apartment_links)

flattened_links = list(itertools.chain(*links))
print(len(flattened_links)) #Counting number of links

100


In [300]:
#Using selenium and beautifulsoup to retrive features + information from apartment-links 
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

for link in flattened_links[5:6]:
    #response = requests.get(link, headers={'name': 'Mathias Rasmussen', 'email': 'kbd655@alumni.ku.dk'})
    test_link=link
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(5)
    area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
    other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")

    ###### NANNAS NYE FEATURES ########
    try:    
        price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold')
    except:
        price = "fejl"
    #Finder label hvor afstand er under
    try:
        school = soup.find('div', string='Afstand').find_next_sibling('div')
    except:
        school = "fejl"
    #Karakter
    try:
        grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div')
    except:
        grades = "fejl"
    #Trivsel
    trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div')
    boligtype = soup.find('span', class_='text-black text-sm pr-2')
    #energi = soup.find('svg', id='Lag_1').find('title')
    energi = soup.find('title', string=re.compile('Energimærke'))
    radon = soup.find('div', class_='h-7 text-sm font-bold')
    
    #kommune_info og liggetid med text behandling
    kommune_info = soup.find('div', class_='sec_municipality_info')
    if kommune_info:
        kommune_text = kommune_info.get_text()
        
        if 'Kommuneskatten er på ' in kommune_text:
            kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
        else:
            kommuneskat = None
        
        if 'grundskyldspromillen på ' in kommune_text:
            grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
        else:
            grundskyldspromillen = None
    liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
    if liggetid:
        liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
    driver.quit()




In [302]:
#Printing features

for i in area_size:
    print(i.get_text())

for i in other:
    print(i.get_text())

print(price.get_text())

print(school.get_text())

print(grades.get_text())

print(trivsel.get_text())

print(boligtype.get_text())

print(energi.get_text())

print(grundskyldspromillen)

print(kommuneskat)

print(radon.get_text())

print(kommune_text)

print(test_link)




120 m²
Grund: 1056 m²
Ejerudg.: 1.680 kr/md
6 værelser
Opført 1870
1 toilet
295.000 kr.
2,63 km
Ikke oplyst
Ikke oplyst
Villa
Energimærke E
27,1‰.
25,7%
Høj
Der er 73.711 indbyggere i Sønderborg Kommune. Kommuneskatten er på 25,7% og grundskyldspromillen på 27,1‰.
https://www.boligsiden.dk/addresses/0a3f50b7-a290-32b8-e044-0003ba298018


In [108]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

for link in flattened_links[:2]:
    try:
        # Initialize the Chrome driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        
        driver.get(link)
        
        # Wait for the necessary elements to load before parsing
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "span.text-gray-800"))
        )
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Scrape the data
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold')
        
        # Find the 'Afstand' label and the related info
        afstand_label = soup.find('div', string='Afstand')
        if afstand_label:
            school = afstand_label.find_next_sibling('div')
        
        # Find the 'Karaktergennemsnit' label and the related info
        karakter_label = soup.find('div', string='Karaktergennemsnit')
        if karakter_label:
            grades = karakter_label.find_next_sibling('div').find_next_sibling('div')
        
        # Find the 'Trivsel' info
        trivsel = soup.find('div', string='Trivsel')
        if trivsel:
            trivsel_info = trivsel.find_next_sibling('div').find_next_sibling('div')
        
        # Handle the 'Liggetid'
        liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
        if liggetid:
            liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')



    except Exception as e:
        print(f"Failed to process {link}: {str(e)}")
    finally:
        # Quit the driver after each iteration
        driver.quit()

